In [1]:
import sys
sys.path.append('../')   # If you move this notebook to another location, change this path to point to the root directory of the solutions project
import pandas as pd
from pathlib import Path
from integrations import integration_base as base
from integrations import elc_integration as elc

# This is an optional step to make floating point numbers display more compactly.
# If you prefer other options, you can change or delete this without harm.
pd.set_option('display.float_format', lambda x: f"{x:03.5f}".rstrip('0'))

In [2]:
# If you are beginning a *new* integration, or starting over

#base.integration_clean()
base.integration_start()

In [3]:
elc.setup(2018)

Integration year set to 2018


# Developer Section

In [5]:
import importlib
importlib.reload(elc)

<module 'integrations.elc_integration' from 'C:\\Working\\solutions\\integrations\\..\\integrations\\elc_integration.py'>

In [2]:
df = pd.DataFrame(elc.get_emissions_factors())
df

,coal,natural gas,large hydro,oil products,onshorewind,offshorewind,solarpvutil,solarpvroof,concentratedsolar,geothermal,waveandtidal,biomass,microwind,instreamhydro,nuclear,wastetoenergy,landfillmethane,biogas
0,1028.25,574.07433,202.26478,828.22,22.4669,14.25182,46.91333,50.37818,41.94167,102.99762,20.59,112.22579,71.0485,24.6888,18.4594,1.6,226.4182,65.72984
1,1250.94861,746.91532,703.94498,1047.55152,43.91821,22.38945,74.86526,79.4291,94.02736,298.52563,36.68999,237.77484,172.09029,65.57254,45.19641,1.6,226.4182,135.43985
2,805.55139,401.23335,0.,608.88848,1.01558,6.11419,18.96141,21.32726,5.,0.,4.49001,4.,0.4,0.5,0.8,1.6,226.4182,2.43098


In [13]:
base.integration_clean()

In [5]:
elc.step1_calculate_TAM()

C:\Working\solutions\integrations\..\model\unitadoption.py:627: RuntimeWarning: invalid value encountered in subtract
  growth_array = np.max([growth.values[1:] - growth.values[:-1], np.full(growth.values[1:].shape, 0.)], axis=0)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\pandas\core\arraylike.py:358: RuntimeWarning: invalid value encountered in minimum
  result = getattr(ufunc, method)(*inputs, **kwargs)
C:\ProgramData\Miniconda3\envs\foo\lib\site-packages\numpy\core\fromnumeric.py:86: RuntimeWarning: invalid value encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
C:\P